# Deep Learning: BlueWins Binary Classification
League of Legends model

In [1]:
# Importing libraries
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# to check the current directory
!pwd

/content


In [3]:
# upload the data to google colab
dataset = pd.read_csv("lol.csv")

# Dropping redundant gameID so it doesn't get used in training
dataset.pop("gameId")
dataset.head()

,blueWins,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueHeralds,...,redTowersDestroyed,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin
0,0,28,2,1,9,6,11,0,0,0,...,0,16567,6.8,17047,197,55,-643,8,19.7,1656.7
1,0,12,1,0,5,5,5,0,0,0,...,1,17620,6.8,17438,240,52,2908,1173,24.0,1762.0
2,0,15,0,0,7,11,4,1,1,0,...,0,17285,6.8,17254,203,28,1172,1033,20.3,1728.5
3,0,43,1,0,4,5,5,1,0,1,...,0,16478,7.0,17961,235,47,1321,7,23.5,1647.8
4,0,75,4,0,6,6,6,0,0,0,...,0,17404,7.0,18313,225,67,1004,-230,22.5,1740.4


# Feature Scaling
Scaling features can lead to more stability during the neural network training, and to achieve that, we can use sklearn StandartScaler.

In [4]:
scaler = StandardScaler()

# Taking labels
labels = dataset['blueWins'].values

# we drop this column since we don't want it to be scaled and used by the NN
dataset.pop("blueWins")

# Scaling our data
features = dataset.values
features = scaler.fit_transform(features)

# Splitting data
Split our data into train and test sets using the `sklearn.train_test_split` method.

In [5]:
train_features, test_features, train_labels, test_labels = train_test_split(features,labels,test_size=0.2,random_state=2001)

In [6]:
from keras.regularizers import l2


# Creating model
We ave used early stopping, L1L2 regualizers (from https://keras.io/api/layers/regularizers/) and `relu` activation function for our setup.

In [7]:
model = Sequential()
model.add(Dense(16,input_dim=38))
model.add(Dense(16, activation='relu',kernel_regularizer='l1_l2',))#we use regulizer to avoid over-fitting
model.add(Dense(1, activation='sigmoid'))
# We're going to use binary_crossentropy as our loss function because we have two states win and lose. I tried different optimizers but Rmsprop proved to be the best for this tas.
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

# Training and Evaluation
Once trained, we can achieve almost training we get almost 73% accuracy which is above the 71% required.

In [8]:
# we can implement early stopping to make perform even better
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=10)
model.fit(train_features, train_labels, epochs=1000, batch_size=32, validation_split=0.33 , callbacks=[early_stopping])

Epoch 1/1000
166/166 [==============================] - 3s 10ms/step - loss: 1.1234 - accuracy: 0.6925 - val_loss: 0.9265 - val_accuracy: 0.7174
Epoch 2/1000
166/166 [==============================] - 1s 7ms/step - loss: 0.8193 - accuracy: 0.7284 - val_loss: 0.7355 - val_accuracy: 0.7258
Epoch 3/1000
166/166 [==============================] - 1s 8ms/step - loss: 0.6766 - accuracy: 0.7303 - val_loss: 0.6378 - val_accuracy: 0.7224
Epoch 4/1000
166/166 [==============================] - 1s 7ms/step - loss: 0.6069 - accuracy: 0.7330 - val_loss: 0.5916 - val_accuracy: 0.7220
Epoch 5/1000
166/166 [==============================] - 1s 8ms/step - loss: 0.5738 - accuracy: 0.7358 - val_loss: 0.5707 - val_accuracy: 0.7235
Epoch 6/1000
166/166 [==============================] - 1s 7ms/step - loss: 0.5585 - accuracy: 0.7348 - val_loss: 0.5618 - val_accuracy: 0.7209
Epoch 7/1000
166/166 [==============================] - 1s 7ms/step - loss: 0.5517 - accuracy: 0.7333 - val_loss: 0.5558 - val_accuracy

In [9]:
loss_and_metrics = model.evaluate(test_features,test_labels)
print("")
print("loss : {}".format(loss_and_metrics[0]))
print("accuracy : {}".format(loss_and_metrics[1]))


62/62 [==============================] - 0s 2ms/step - loss: 0.5392 - accuracy: 0.7313

loss : 0.5392405986785889
accuracy : 0.7312753200531006
